In [2]:
from pyedflib import highlevel
import pyedflib as plib
import numpy as np
import matplotlib.pyplot as plt
import statistics as stat

def read_file_signal(file_path, index):
    f = plib.EdfReader(file_path)
    signal = f.readSignal(index)
    f.close()
    return signal

def signal_labels(file_path):
    f = plib.EdfReader(file_path)
    labels = f.getSignalLabels()
    f.close()
    return labels

def combined_signal(file_list, index):
    combined_signals = []
    for file in file_list:
        temp = read_file_signal(file, index)
        combined_signals.append(temp)
    return np.concatenate(combined_signals)

edf_files_severe = [
    "c:\\Users\\anany\\Downloads\\00000066-297469[001].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[002].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[003].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[004].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[005].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[006].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[007].edf"
]

edf_files_normal = [
    "C:\\Users\\anany\\Downloads\\00000064-297469[001].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[002].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[003].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[004].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[005].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[006].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[007].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[008].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[009].edf"
]

edf_files_moderate = [
    "C:\\Users\\anany\\Downloads\\00000074-297469[001].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[002].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[003].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[004].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[005].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[006].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[007].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[008].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[009].edf"
]

edf_files_mild = [
    "C:\\Users\\anany\\Downloads\\00000067-297469[001].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[002].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[003].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[004].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[005].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[006].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[007].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[008].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[009].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[010].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[011].edf",
]

labels = signal_labels(edf_files_normal[0])
print(labels)
labels[16] = "Flow patient 1"
labels[17] = "Flow patient 2"
labels[23] = "Flow patient 3"
cv_dict = {}

['EEG F3-A2', 'EEG F4-A1', 'EEG A1-A2', 'EEG C3-A2', 'EEG C4-A1', 'EEG O1-A2', 'EEG O2-A1', 'EOG LOC-A2', 'EOG ROC-A2', 'EMG Chin', 'Leg 1', 'Leg 2', 'ECG I', 'RR', 'ECG II', 'Snore', 'Flow Patient', 'Flow Patient', 'Effort THO', 'Effort ABD', 'SpO2', 'Pleth', 'Body', 'Flow Patient', 'xPAP CPAP', 'xPAP IPAP', 'xPAP EPAP', 'Leak Total', 'PulseRate', 'PressCheck', 'ECG IIHF', 'Technical']


In [15]:
def plot_signal(label, signal):
    plt.figure(figsize=(20, 10))
    plt.plot(signal)
    plt.title(f"{label} signal")
    plt.xlabel("time")
    plt.ylabel("amplitude")
    # plt.ylim(-300, 300)
    plt.show()

def thresholding(signal, threshold):
    mean = signal.mean()
    std = np.std(signal)
    lower_limit = mean - threshold * std
    upper_limit = mean + threshold * std
    filtered_signal = np.where((signal >= lower_limit) & (signal <= upper_limit), signal, mean)
    return filtered_signal

def resample_signal(signal, num_samples):
    original_indices = np.linspace(0, len(signal) - 1, num=len(signal))
    resampled_indices = np.linspace(0, len(signal) - 1, num=num_samples)
    resampled_signal = np.interp(resampled_indices, original_indices, signal)
    return resampled_signal

In [33]:
def count_spo2_drops(spo2_data, timestamps, drop=0.03, duration=10):
    mean_spo2 = np.mean(spo2_data)  
    drop_start = None
    count = 0

    for i in range(1, len(spo2_data)):
        if (mean_spo2 - spo2_data[i])/mean_spo2 >= drop:
            if drop_start is None:
                drop_start = timestamps[i]  # Start timing the drop
        else:
            if drop_start is not None and (timestamps[i - 1] - drop_start) >= duration:
                count += 1  # Count the drop
            drop_start = None  # Reset the drop

    if drop_start is not None and (timestamps[-1] - drop_start) >= duration:
        count += 1

    return count

In [5]:
spo2_index = labels.index("SpO2")
pulse_index = labels.index("PulseRate")
npt_index = labels.index("Flow patient 1")
therm_index = labels.index("Flow patient 2")    

print("SpO2 index: ", spo2_index)
print("Pulse index: ", pulse_index)
print("Flow patient 1 index: ", npt_index)
print("Flow patient 2 index: ", therm_index)

SpO2 index:  20
Pulse index:  28
Flow patient 1 index:  16
Flow patient 2 index:  17


# NORMAL SPO2

In [12]:
def count_spo2_drops(spo2_data, timestamps, drop=0.03, duration=10):
    mean_spo2 = np.mean(spo2_data)  
    drop_start = None
    count = 0

    for i in range(1, len(spo2_data)):
        if (mean_spo2 - spo2_data[i])/mean_spo2 >= drop:
            if drop_start is None:
                drop_start = timestamps[i]  # Start timing the drop
        else:
            if drop_start is not None and (timestamps[i - 1] - drop_start) >= duration:
                count += 1  # Count the drop
            drop_start = None  # Reset the drop

    if drop_start is not None and (timestamps[-1] - drop_start) >= duration:
        count += 1

    return count


In [47]:
for j in range(9):
    spo2_data = read_file_signal(edf_files_normal[j], spo2_index)
    spo2_denoised = thresholding(spo2_data, 2)
    timestamps = np.linspace(0, 3600, len(spo2_denoised))
    # print(len(spo2_denoised))
    count = count_spo2_drops(spo2_denoised, timestamps, 0.03, 10)
    print(f"Number of SpO2 drops in hour {j + 1}: {count}")


Number of SpO2 drops in hour 1: 0
Number of SpO2 drops in hour 2: 0
Number of SpO2 drops in hour 3: 0
Number of SpO2 drops in hour 4: 0
Number of SpO2 drops in hour 5: 0
Number of SpO2 drops in hour 6: 0
Number of SpO2 drops in hour 7: 1
Number of SpO2 drops in hour 8: 0
Number of SpO2 drops in hour 9: 0


In [51]:
combined_spo2 = combined_signal(edf_files_normal, spo2_index)
combined_spo2_denoised = thresholding(combined_spo2, 2)
mean = np.mean(combined_spo2_denoised)
timestamps = np.linspace(0, 32400, len(combined_spo2_denoised))
count = count_spo2_drops(combined_spo2_denoised, timestamps, 0.03, 10)
print(f"Number of SpO2 drops in overall in normal: {count}")

Number of SpO2 drops in overall in normal: 2


In [24]:
for j in range(11):
    spo2_data = read_file_signal(edf_files_mild[j], spo2_index)
    spo2_denoised = thresholding(spo2_data, 2)
    timestamps = np.linspace(0, 3600, len(spo2_denoised))
    # print(len(spo2_denoised))
    count = count_spo2_drops(spo2_denoised, timestamps, 0.03, 10)
    print(f"Number of SpO2 drops in hour {j + 1}: {count}")

Number of SpO2 drops in hour 1: 2
Number of SpO2 drops in hour 2: 0
Number of SpO2 drops in hour 3: 0
Number of SpO2 drops in hour 4: 0
Number of SpO2 drops in hour 5: 0
Number of SpO2 drops in hour 6: 0
Number of SpO2 drops in hour 7: 0
Number of SpO2 drops in hour 8: 0
Number of SpO2 drops in hour 9: 0
Number of SpO2 drops in hour 10: 0
Number of SpO2 drops in hour 11: 0


In [44]:
combined_spo2 = combined_signal(edf_files_mild, spo2_index)
combined_spo2_denoised = thresholding(combined_spo2, 2)
mean = np.mean(combined_spo2_denoised)
timestamps = np.linspace(0, 39600, len(combined_spo2_denoised))
count = count_spo2_drops(combined_spo2_denoised, timestamps, 0.03, 10)
print(f"Number of SpO2 drops in 7 hours: {count}")

Number of SpO2 drops in 7 hours: 23


In [34]:
for j in range(9):
    spo2_data = read_file_signal(edf_files_moderate[j], spo2_index)
    spo2_denoised = thresholding(spo2_data, 2)
    timestamps = np.linspace(0, 3600, len(spo2_denoised))
    # print(len(spo2_denoised))
    count = count_spo2_drops(spo2_denoised, timestamps, 0.03, 10)
    print(f"Number of SpO2 drops in hour {j + 1}: {count}")

Number of SpO2 drops in hour 1: 1
Number of SpO2 drops in hour 2: 0
Number of SpO2 drops in hour 3: 6
Number of SpO2 drops in hour 4: 0
Number of SpO2 drops in hour 5: 0
Number of SpO2 drops in hour 6: 0
Number of SpO2 drops in hour 7: 0
Number of SpO2 drops in hour 8: 0
Number of SpO2 drops in hour 9: 0


In [45]:
combined_spo2 = combined_signal(edf_files_moderate, spo2_index)
combined_spo2_denoised = thresholding(combined_spo2, 2)
mean = np.mean(combined_spo2_denoised)
timestamps = np.linspace(0, 32400, len(combined_spo2_denoised))
count = count_spo2_drops(combined_spo2_denoised, timestamps, 0.03, 10)
print(f"Number of SpO2 drops in 7 hours: {count}")

Number of SpO2 drops in 7 hours: 31


In [49]:
for j in range(7):
    spo2_data = read_file_signal(edf_files_severe[j], spo2_index)
    spo2_denoised = thresholding(spo2_data, 2)
    timestamps = np.linspace(0, 3600, len(spo2_denoised))
    # print(len(spo2_denoised))
    count = count_spo2_drops(spo2_denoised, timestamps, 0.03, 10)
    print(f"Number of SpO2 drops in hour {j + 1}: {count}")

Number of SpO2 drops in hour 1: 29
Number of SpO2 drops in hour 2: 42
Number of SpO2 drops in hour 3: 43
Number of SpO2 drops in hour 4: 37
Number of SpO2 drops in hour 5: 0
Number of SpO2 drops in hour 6: 0
Number of SpO2 drops in hour 7: 8


In [50]:
combined_spo2 = combined_signal(edf_files_severe, spo2_index)
combined_spo2_denoised = thresholding(combined_spo2, 2)
mean = np.mean(combined_spo2_denoised)
timestamps = np.linspace(0, 25200, len(combined_spo2_denoised))
count = count_spo2_drops(combined_spo2_denoised, timestamps, 0.03, 10)
print(f"Number of SpO2 drops in 7 hours: {count}")

Number of SpO2 drops in 7 hours: 196


# THERMISTOR

In [144]:
def count_ob_apnea(fp2_signal, timestamps):
    mean_fp2 = np.mean(fp2_signal)
    count = 0
    apnea_start = None
    time_drops = []
    for i in range(1, len(fp2_signal)):
        if (mean_fp2 - fp2_signal[i])/mean_fp2 >= 0.9:
            if apnea_start is None:
                apnea_start = timestamps[i]
        else:
            if apnea_start is not None and (timestamps[i - 1] - apnea_start) >= 1000:
                count += 1  # Count the drop
                time_drops.append((apnea_start, i))
            apnea_start = None  # Reset the drop

    if apnea_start is not None and (timestamps[-1] - apnea_start) >= 1000:
        count += 1
        time_drops.append((apnea_start, timestamps[-1]))

    return count, time_drops


In [145]:
for j in range(9):
    fp2_data = read_file_signal(edf_files_normal[j], therm_index)
    fp2_denoised = thresholding(fp2_data, 2)
    timestamps = np.linspace(0, 360000, len(fp2_denoised))
    # print(len(fp2_denoised))
    count, time_drops = count_ob_apnea(fp2_denoised, timestamps)
    print(f"Number of fp2 drops in hour {j + 1}: {count}")
    print(f"Time of drops: {time_drops}")

Number of fp2 drops in hour 1: 0
Time of drops: []
Number of fp2 drops in hour 2: 0
Time of drops: []
Number of fp2 drops in hour 3: 0
Time of drops: []
Number of fp2 drops in hour 4: 0
Time of drops: []
Number of fp2 drops in hour 5: 0
Time of drops: []
Number of fp2 drops in hour 6: 0
Time of drops: []
Number of fp2 drops in hour 7: 0
Time of drops: []
Number of fp2 drops in hour 8: 0
Time of drops: []
Number of fp2 drops in hour 9: 3
Time of drops: [(33814.916120461814, 21016), (134741.58091135108, 81799), (137984.58479974538, 83730)]


In [147]:
for j in range(11):
    fp2_data = read_file_signal(edf_files_mild[j], therm_index)
    fp2_denoised = thresholding(fp2_data, 2)
    timestamps = np.linspace(0, 360000, len(fp2_denoised))
    # print(len(fp2_denoised))
    count, time_drops = count_ob_apnea(fp2_denoised, timestamps)
    print(f"Number of fp2 drops in hour {j + 1}: {count}")
    print(f"Time of drops: {time_drops}")

Number of fp2 drops in hour 1: 0
Time of drops: []
Number of fp2 drops in hour 2: 0
Time of drops: []
Number of fp2 drops in hour 3: 0
Time of drops: []
Number of fp2 drops in hour 4: 0
Time of drops: []
Number of fp2 drops in hour 5: 0
Time of drops: []
Number of fp2 drops in hour 6: 0
Time of drops: []
Number of fp2 drops in hour 7: 2
Time of drops: [(286194.7949855416, 289869), (306681.851894033, 310715)]
Number of fp2 drops in hour 8: 0
Time of drops: []
Number of fp2 drops in hour 9: 0
Time of drops: []
Number of fp2 drops in hour 10: 0
Time of drops: []
Number of fp2 drops in hour 11: 57
Time of drops: [(119.76751012739975, 253), (3212.5873304761344, 672), (6023.601244642752, 1051), (15147.067457288793, 2297), (18916.22145247461, 2872), (21234.07503082252, 3222), (27208.3602418834, 4062), (38586.27370398638, 5690), (41207.06863148007, 6044), (44426.70110961075, 6512), (47597.017554159575, 6953), (50619.38589796278, 7343), (53599.48335583867, 7794), (59136.96941231727, 8589), (653

In [148]:
for j in range(9):
    fp2_data = read_file_signal(edf_files_moderate[j], therm_index)
    fp2_denoised = thresholding(fp2_data, 2)
    timestamps = np.linspace(0, 360000, len(fp2_denoised))
    # print(len(fp2_denoised))
    count, time_drops = count_ob_apnea(fp2_denoised, timestamps)
    print(f"Number of fp2 drops in hour {j + 1}: {count}")
    print(f"Time of drops: {time_drops}")

Number of fp2 drops in hour 1: 0
Time of drops: []
Number of fp2 drops in hour 2: 0
Time of drops: []
Number of fp2 drops in hour 3: 2
Time of drops: [(144789.40219278386, 145818), (159292.4424790069, 161340)]
Number of fp2 drops in hour 4: 4
Time of drops: [(179180.49772360479, 181214), (184985.51384864957, 187275), (190476.52910146973, 192094), (242186.6727407576, 243313)]
Number of fp2 drops in hour 5: 0
Time of drops: []
Number of fp2 drops in hour 6: 0
Time of drops: []
Number of fp2 drops in hour 7: 0
Time of drops: []
Number of fp2 drops in hour 8: 0
Time of drops: []
Number of fp2 drops in hour 9: 0
Time of drops: []


In [149]:
for j in range(7):
    fp2_data = read_file_signal(edf_files_severe[j], therm_index)
    fp2_denoised = thresholding(fp2_data, 2)
    timestamps = np.linspace(0, 360000, len(fp2_denoised))
    # print(len(fp2_denoised))
    count, time_drops = count_ob_apnea(fp2_denoised, timestamps)
    print(f"Number of fp2 drops in hour {j + 1}: {count}")
    print(f"Time of drops: {time_drops}")

Number of fp2 drops in hour 1: 27
Time of drops: [(139891.3885871905, 140991), (149562.41545115403, 150673), (160550.44597346103, 161859), (163518.45421792837, 164637), (172171.47825410627, 173252), (184506.51251809034, 186264), (187926.52201811672, 189969), (217620.60450167916, 219001), (230230.63952955426, 231410), (239534.6653740705, 240632), (242532.6737018714, 243617), (245997.68332689814, 248398), (271298.7536076489, 272534), (274350.76208545023, 275406), (280209.77836049546, 281693), (286858.7968299912, 288334), (290609.80724946456, 291725), (293790.81608560024, 295294), (296936.8248245134, 298732), (300352.83431342867, 302686), (309467.8596329434, 310481), (318270.88408578915, 319452), (323220.8978358273, 324526), (326681.9074497429, 327847), (330701.9186164406, 331959), (343401.9538943164, 345480), (351849.9773610482, 353092)]
Number of fp2 drops in hour 2: 65
Time of drops: [(4028.0111889199693, 5469), (7809.021691726922, 9233), (20908.058077939106, 22115), (26888.07468909636

In [150]:
combined_fp2 = combined_signal(edf_files_severe, therm_index)
combined_fp2_denoised = thresholding(combined_fp2, 2)
mean = np.mean(combined_fp2_denoised)
timestamps = np.linspace(0, 2520000, len(combined_fp2_denoised))
count, time_drops = count_ob_apnea(combined_fp2_denoised, timestamps)
print(f"Number of fp2 drops in 7 hours: {count}")
print(f"Time of drops: {time_drops}")

Number of fp2 drops in 7 hours: 74
Time of drops: [(109790.0435674776, 110883), (125218.04968970224, 126231), (151135.05997423013, 152201), (262570.10419448576, 263823), (285727.1133837751, 286916), (302615.1200853651, 303637), (332991.1321393381, 334358), (339815.1348472757, 341891), (356862.14161196095, 358293), (370773.1471322012, 371784), (399568.158558793, 401323), (429489.17043221043, 430922), (520648.20660643117, 521851), (564100.2238492952, 565321), (588432.2335048545, 589539), (630895.250355258, 632117), (677569.2688766939, 678803), (682194.2707120121, 683614), (750287.2977330546, 753603), (764494.3033707553, 765912), (774441.307317979, 775688), (780655.309783853, 781938), (796193.3159497285, 798777), (798786.3169786971, 800308), (813321.3227465566, 816894), (822561.3264132247, 823838), (831355.3299029086, 832521), (833667.3308203693, 835268), (839299.3330552909, 841770), (850520.3375080704, 851521), (860804.3415890244, 863088), (868694.3447199779, 872591), (876065.3476449791,

In [151]:
combined_fp2 = combined_signal(edf_files_normal, therm_index)
combined_fp2_denoised = thresholding(combined_fp2, 2)
mean = np.mean(combined_fp2_denoised)
timestamps = np.linspace(0, 3240000, len(combined_fp2_denoised))
count, time_drops = count_ob_apnea(combined_fp2_denoised, timestamps)
print(f"Number of fp2 drops in 9 hours: {count}")
print(f"Time of drops: {time_drops}")

Number of fp2 drops in 9 hours: 0
Time of drops: []


In [152]:
combined_fp2 = combined_signal(edf_files_moderate, therm_index)
combined_fp2_denoised = thresholding(combined_fp2, 2)
mean = np.mean(combined_fp2_denoised)
timestamps = np.linspace(0, 3240000, len(combined_fp2_denoised))
count, time_drops = count_ob_apnea(combined_fp2_denoised, timestamps)
print(f"Number of fp2 drops in 9 hours: {count}")
print(f"Time of drops: {time_drops}")

Number of fp2 drops in 9 hours: 4
Time of drops: [(956075.4131402823, 881329), (1369133.6898898215, 1261215), (1375445.592135577, 1267274), (1437642.5658240707, 1323206)]


In [153]:
combined_fp2 = combined_signal(edf_files_mild, therm_index)
combined_fp2_denoised = thresholding(combined_fp2, 2)
mean = np.mean(combined_fp2_denoised)
timestamps = np.linspace(0, 3960000, len(combined_fp2_denoised))
count, time_drops = count_ob_apnea(combined_fp2_denoised, timestamps)
print(f"Number of fp2 drops in 11 hours: {count}")
print(f"Time of drops: {time_drops}")

Number of fp2 drops in 11 hours: 4
Time of drops: [(1049.8975787838128, 7893), (2474188.8510829206, 2282123), (2653154.0886730268, 2449869), (2675374.390012432, 2470715)]


# PRESSURE TRANSDUCER + SPO2 DROP COUNT

In [154]:
def count_ob_hypopnea(fp1_data, spo2_data, timestamps):
    mean_fp1 = np.mean(fp1_data)  
    mean_spo2 = np.mean(fp1_data)
    drop_start = None
    count = 0
    time_drops = []

    for i in range(1, min(len(fp1_data), len(spo2_data))):
        if (mean_fp1 - fp1_data[i])/mean_fp1 >= 0.3 and (mean_spo2 - spo2_data[i])/mean_spo2 >= 0.03:
            if drop_start is None:
                drop_start = timestamps[i]  # Start timing the drop
        else:
            if drop_start is not None and (timestamps[i - 1] - drop_start) >= 10:
                count += 1  # Count the drop
                time_drops.append((drop_start, i))
            drop_start = None  # Reset the drop

    if drop_start is not None and (timestamps[-1] - drop_start) >= 10:
        count += 1
        time_drops.append((drop_start, timestamps[-1]))

    return count, time_drops


In [155]:
for j in range(7):
    fp1_big = read_file_signal(edf_files_severe[j], npt_index)
    fp1_small = resample_signal(fp1_big, 3600)
    fp1_denoised = thresholding(fp1_small, 2)
    spo2_data = read_file_signal(edf_files_severe[j], spo2_index)
    spo2_denoised = thresholding(spo2_data, 2)
    timestamps = np.linspace(0, 3600, len(fp1_denoised))
    # print(len(fp1_denoised))
    count, time_drops = count_ob_hypopnea(fp1_denoised, spo2_denoised, timestamps)
    print(f"Number of fp1 drops in hour {j + 1}: {count}")
    print(f"Time of drops: {time_drops}")

Number of fp1 drops in hour 1: 25
Time of drops: [(51.01417060294526, 63), (66.01833842734094, 77), (256.07113086968604, 271), (287.0797443734371, 305), (341.0947485412615, 355), (371.1030841900528, 388), (938.260627952209, 952), (998.2772992497917, 1020), (1606.4462350652962, 1623), (1845.512642400667, 1857), (1885.5237565990553, 1897), (2030.5640455682135, 2043), (2119.5887746596277, 2130), (2178.605168102251, 2189), (2222.617393720478, 2238), (2331.6476799110865, 2344), (2424.6735204223396, 2437), (2796.776882467352, 2809), (2862.795220894693, 2877), (2939.816615726591, 2951), (3009.8360655737706, 3024), (3142.873020283412, 3153), (3308.9191442067245, 3320), (3404.9458182828566, 3417), (3431.953320366769, 3442)]
Number of fp1 drops in hour 2: 0
Time of drops: []
Number of fp1 drops in hour 3: 0
Time of drops: []
Number of fp1 drops in hour 4: 15
Time of drops: [(239.06640733537097, 250), (401.11141983884414, 412), (1646.4573492636846, 1658), (1788.4968046679635, 1799), (1918.5329258

In [156]:
for j in range(9):
    fp1_big = read_file_signal(edf_files_normal[j], npt_index)
    fp1_small = resample_signal(fp1_big, 3600)
    fp1_denoised = thresholding(fp1_small, 2)
    spo2_data = read_file_signal(edf_files_normal[j], spo2_index)
    spo2_denoised = thresholding(spo2_data, 2)
    timestamps = np.linspace(0, 3600, len(fp1_denoised))
    # print(len(fp1_denoised))
    count, time_drops = count_ob_hypopnea(fp1_denoised, spo2_denoised, timestamps)
    print(f"Number of fp1 drops in hour {j + 1}: {count}")
    print(f"Time of drops: {time_drops}")

Number of fp1 drops in hour 1: 0
Time of drops: []
Number of fp1 drops in hour 2: 0
Time of drops: []
Number of fp1 drops in hour 3: 0
Time of drops: []
Number of fp1 drops in hour 4: 1
Time of drops: [(1871.5198666296194, 1884)]
Number of fp1 drops in hour 5: 0
Time of drops: []
Number of fp1 drops in hour 6: 0
Time of drops: []
Number of fp1 drops in hour 7: 0
Time of drops: []
Number of fp1 drops in hour 8: 0
Time of drops: []
Number of fp1 drops in hour 9: 0
Time of drops: []


In [157]:
for j in range(11):
    fp1_big = read_file_signal(edf_files_mild[j], npt_index)
    fp1_small = resample_signal(fp1_big, 3600)
    fp1_denoised = thresholding(fp1_small, 2)
    spo2_data = read_file_signal(edf_files_mild[j], spo2_index)
    spo2_denoised = thresholding(spo2_data, 2)
    timestamps = np.linspace(0, 3600, len(fp1_denoised))
    # print(len(fp1_denoised))
    count, time_drops = count_ob_hypopnea(fp1_denoised, spo2_denoised, timestamps)
    print(f"Number of fp1 drops in hour {j + 1}: {count}")
    print(f"Time of drops: {time_drops}")

Number of fp1 drops in hour 1: 8
Time of drops: [(38.01055848846902, 50), (182.05056960266742, 194), (216.0600166712976, 231), (240.06668519033067, 254), (290.0805779383162, 305), (353.098082800778, 365), (446.12392331203114, 459), (2102.5840511253127, 2115)]
Number of fp1 drops in hour 2: 0
Time of drops: []
Number of fp1 drops in hour 3: 0
Time of drops: []
Number of fp1 drops in hour 4: 0
Time of drops: []
Number of fp1 drops in hour 5: 0
Time of drops: []
Number of fp1 drops in hour 6: 0
Time of drops: []
Number of fp1 drops in hour 7: 9
Time of drops: [(2789.7749374826344, 2801), (2859.794387329814, 2871), (2985.8293970547375, 3006), (3052.848013337038, 3064), (3164.879133092526, 3179), (3183.8844123367603, 3194), (3316.921367046402, 3331), (3511.9755487635457, 3524), (3572.9924979160883, 3583)]
Number of fp1 drops in hour 8: 0
Time of drops: []
Number of fp1 drops in hour 9: 22
Time of drops: [(268.07446512920257, 279), (713.198110586274, 726), (835.2320088913588, 849), (943.2620

In [158]:
for j in range(9):
    fp1_big = read_file_signal(edf_files_moderate[j], npt_index)
    fp1_small = resample_signal(fp1_big, 3600)
    fp1_denoised = thresholding(fp1_small, 2)
    spo2_data = read_file_signal(edf_files_moderate[j], spo2_index)
    spo2_denoised = thresholding(spo2_data, 2)
    timestamps = np.linspace(0, 3600, len(fp1_denoised))
    # print(len(fp1_denoised))
    count, time_drops = count_ob_hypopnea(fp1_denoised, spo2_denoised, timestamps)
    print(f"Number of fp1 drops in hour {j + 1}: {count}")
    print(f"Time of drops: {time_drops}")

Number of fp1 drops in hour 1: 0
Time of drops: []
Number of fp1 drops in hour 2: 0
Time of drops: []
Number of fp1 drops in hour 3: 1
Time of drops: [(1897.527090858572, 1914)]
Number of fp1 drops in hour 4: 18
Time of drops: [(197.0547374270631, 209), (225.062517365935, 236), (1802.5006946373994, 1814), (1815.5043067518757, 1827), (2012.5590441789388, 2024), (2126.590719644346, 2145), (2180.6057238121703, 2191), (2218.6162823006393, 2230), (2241.622672964713, 2252), (2271.631008613504, 2286), (2550.7085301472634, 2562), (2757.7660461239234, 2770), (2819.7832731314256, 2833), (3089.8582939705475, 3101), (3227.896637954988, 3238), (3405.9460961378163, 3418), (3547.9855515420954, 3558), (3585.9961100305645, 3597)]
Number of fp1 drops in hour 5: 15
Time of drops: [(767.2131147540985, 779), (906.2517365934983, 917), (1034.2873020283414, 1045), (1365.3792720200056, 1376), (1463.4065018060573, 1480), (1622.4506807446514, 1634), (2073.575993331481, 2085), (2287.6354542928593, 2304), (2460.68

In [130]:
def count_ob_hypopnea_2(fp1_data, spo2_data, timestamps):   ##gives wrong results
    mean_fp1 = np.mean(fp1_data)  
    mean_spo2 = np.mean(fp1_data)
    resampled_fp1 = fp1_data.reshape(-1, 100).mean(axis=1)
    drop_start = None
    count = 0
    time_drops = []

    for i in range(1, len(spo2_data)):  ##continue from here
        if (mean_fp1 - resampled_fp1[i])/mean_fp1 >= 0.3 and (mean_spo2 - spo2_data[i])/mean_spo2 >= 0.03:
            if drop_start is None:
                drop_start = timestamps[i]  # Start timing the drop
        else:
            if drop_start is not None and (timestamps[i - 1] - drop_start) >= 10:
                count += 1  # Count the drop
                time_drops.append((drop_start, i))
            drop_start = None  # Reset the drop

    if drop_start is not None and (timestamps[-1] - drop_start) >= 10:
        count += 1
        time_drops.append((drop_start, len(spo2_data)))

    return count, time_drops

In [159]:
spo2_data = combined_signal(edf_files_normal, spo2_index)
spo2_denoised = thresholding(spo2_data, 2)
fp1_data = combined_signal(edf_files_normal, npt_index)
fp1_denoised = thresholding(fp1_data, 2)
fp1_small = resample_signal(fp1_denoised, 32400)
timestamps = np.linspace(0, 32400, len(fp1_denoised))
count, time_drops = count_ob_hypopnea(fp1_small, spo2_denoised, timestamps)
print(f"Number of fp1 drops in 9 hours: {count}")
print(f"Time of drops: {time_drops}")

Number of fp1 drops in 9 hours: 1
Time of drops: [(323.98964220797023, 32400.0)]


In [160]:
spo2_data = combined_signal(edf_files_mild, spo2_index)
spo2_denoised = thresholding(spo2_data, 2)
fp1_data = combined_signal(edf_files_mild, npt_index)
fp1_denoised = thresholding(fp1_data, 2)
fp1_small = resample_signal(fp1_denoised, 39600)
timestamps = np.linspace(0, 39600, len(fp1_denoised))
count, time_drops = count_ob_hypopnea(fp1_small, spo2_denoised, timestamps)
print(f"Number of fp1 drops in 9 hours: {count}")
print(f"Time of drops: {time_drops}")

Number of fp1 drops in 9 hours: 3
Time of drops: [(291.7261898403741, 29481), (356.24933752823466, 34634), (381.72670749273027, 39600.0)]


In [161]:
spo2_data = combined_signal(edf_files_moderate, spo2_index)
spo2_denoised = thresholding(spo2_data, 2)
fp1_data = combined_signal(edf_files_moderate, npt_index)
fp1_denoised = thresholding(fp1_data, 2)
fp1_small = resample_signal(fp1_denoised, 32400)
timestamps = np.linspace(0, 32400, len(fp1_denoised))
count, time_drops = count_ob_hypopnea(fp1_small, spo2_denoised, timestamps)
print(f"Number of fp1 drops in 9 hours: {count}")
print(f"Time of drops: {time_drops}")

Number of fp1 drops in 9 hours: 1
Time of drops: [(323.98923551554986, 32400.0)]


In [162]:
spo2_data = combined_signal(edf_files_severe, spo2_index)
spo2_denoised = thresholding(spo2_data, 2)
fp1_data = combined_signal(edf_files_severe, npt_index)
fp1_denoised = thresholding(fp1_data, 2)
fp1_small = resample_signal(fp1_denoised, 25200)
timestamps = np.linspace(0, 25200, len(fp1_denoised))
count, time_drops = count_ob_hypopnea(fp1_small, spo2_denoised, timestamps)
print(f"Number of fp1 drops in 9 hours: {count}")
print(f"Time of drops: {time_drops}")

Number of fp1 drops in 9 hours: 0
Time of drops: []
